# Download and store STOOQ data

This notebook contains information on downloading the STOOQ stock and ETF price data that we use in [Chapter 09](../09_time_series_models) for a pairs trading strategy based on cointegration and [Chapter 11](../11_decision_trees_random_forests) for a long-short strategy using Random Forest return predictions.

## Imports & Settings

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify the path to the `DATA_STORE` if you would like to store the data elsewhere and change the notebooks accordingly

In [15]:
DATA_STORE = Path('assets.h5')

## Stooq Historical Market Data

> Note that the below downloading details may change at any time as Stooq updates their website; if you encounter errors, please inspect their website and raise a GitHub issue to let us know so we can update the information.

> Update 12/2020: please note that STOOQ will disable automatic downloads and require CAPTCHA starting Dec 10, 2020 so that the code that downloads and unpacks the zip files will no longer work; please navigate to their website [here](https://stooq.com/db/h/) for manual download.

### Download price data

1. Download **price data** for the selected combination of asset class, market and frequency from [the Stooq website](https://stooq.com/db/h/)
2. Store the result under `stooq` using the preferred folder structure outlined on the website. It has the structure: `/data/freq/market/asset_class`, such as `/data/daily/us/nasdaq etfs`.

In [16]:
stooq_path = Path('stooq') 
if not stooq_path.exists():
    stooq_path.mkdir()

Use the symbol for the market you want to download price data for. In this book we'll be useing `us` and `jp`. 

In [17]:
STOOQ_URL = 'https://static.stooq.com/db/h/'

In [18]:
def download_price_data(market='us'):
    data_url = f'd_{market}_txt.zip'
    response = requests.get(STOOQ_URL + data_url).content
    with ZipFile(BytesIO(response)) as zip_file:
        for i, file in enumerate(zip_file.namelist()):
            if not file.endswith('.txt'):
                continue
            local_file = stooq_path / file
            local_file.parent.mkdir(parents=True, exist_ok=True)
            with local_file.open('wb') as output:
                for line in zip_file.open(file).readlines():
                    output.write(line)

In [19]:
# for market in ['us', 'jp']:
#     download_price_data(market=market)

### Add symbols

Add the corresponding **symbols**, i.e., tickers and names by following the directory tree on the same site. You can also adapt the following code snippet using the appropriate asset code that you find by inspecting the url; this example works for NASDAQ ETFs that have code `g=69`:
```python
df = pd.read_csv('https://stooq.com/db/l/?g=69', sep='        ').apply(lambda x: x.str.strip())
df.columns = ['ticker', 'name']
df.drop_duplicates('ticker').to_csv('stooq/data/tickers/us/nasdaq etfs.csv', index=False)
```

To resolve the `EmptyDataError`, the issue stems from attempting to automatically fetch symbol lists (tickers and names) from Stooq URLs, which are now protected by CAPTCHA and cannot be reliably accessed via scripts or tools. You must manually download each symbol list in a browser, save it as a plain text file (e.g., `.txt`), and then modify the script to read from these local files using an appropriate separator. Based on typical Stooq symbol list formats, these files are often delimited by tabs (`\t`) or multiple spaces (not commas), with no header row—e.g., lines like `133A\tNEXT FUNDS JPX Prime 150 Index Exchange Traded Fund` or with spaces instead of tabs.

### Step 1: Manually Download Symbol Lists
1. Visit https://stooq.com/db/h/ in your browser to navigate the site structure.
2. For each market/asset class in `metadata_dict`, access the symbol list URL (replace `{code}` with the value from `metadata_dict`):
   - URL format: `https://stooq.com/db/l/?g={code}`
   - Examples:
     - JP TSE ETFs (code 34): https://stooq.com/db/l/?g=34
     - JP TSE Stocks (code 32): https://stooq.com/db/l/?g=32
     - US NASDAQ ETFs (code 69): https://stooq.com/db/l/?g=69
     - US NASDAQ Stocks (code 27): https://stooq.com/db/l/?g=27
     - US NYSE ETFs (code 70): https://stooq.com/db/l/?g=70
     - US NYSE Stocks (code 28): https://stooq.com/db/l/?g=28
     - US NYSEMKT Stocks (code 26): https://stooq.com/db/l/?g=26
3. If prompted, solve any CAPTCHA.
4. The page should display plain text with ticker-name pairs (one per line, delimited by tabs or spaces).
5. Select all text (Ctrl+A or Cmd+A), copy it, and paste into a new file.
6. Save the file in the appropriate directory with a `.txt` extension:
   - Example: `stooq/tickers/jp/tse etfs.txt` (note: use lowercase and spaces as in `asset_class`).
7. Repeat for all entries in `metadata_dict`.
8. Verify a sample file's content in a text editor (e.g., `head -n 5 stooq/tickers/jp/tse\ etfs.txt`). It should look like ticker-delimiter-name, without headers. If it's comma-separated (unlikely), note that for Step 2.

### Step 2: Modify the Script
Replace the original symbol-fetching loop with this updated version. It reads from your manually saved `.txt` files using `sep='\s+'` (handles tabs or multiple spaces), cleans the data, and saves a comma-separated `.csv` version (which the rest of the script expects in `get_stooq_prices_and_tickers`).

```python
for (market, asset_class), code in metadata_dict.items():
    # Path to your manually downloaded .txt file
    path_txt = stooq_path / 'tickers' / market / f'{asset_class}.txt'
    
    # Read with flexible separator (\s+ handles tabs or multiple spaces); no header
    df = pd.read_csv(path_txt, sep='\s+', header=None, engine='python')
    df.columns = ['ticker', 'name']
    df = df.apply(lambda x: x.str.strip())  # Strip any extra whitespace
    df = df.drop_duplicates('ticker').dropna()
    
    print(market, asset_class, f'# tickers: {df.shape[0]:,.0f}')
    
    # Ensure directory exists
    path_txt.parent.mkdir(parents=True, exist_ok=True)
    
    # Save as .csv (comma-separated) for the rest of the script
    path_csv = path_txt.with_suffix('.csv')
    df.to_csv(path_csv, index=False)
```

- **Why this works**:
  - `sep='\s+'`: Uses regex to split on one or more whitespace characters (tabs, spaces). If your files are strictly tab-separated, you can change to `sep='\t'`. If comma-separated (rare for symbols), use `sep=','`.
  - `header=None`: Symbol lists typically have no header row.
  - `engine='python'`: Helps with variable delimiters.
  - Saves to `.csv`: Matches what `get_stooq_prices_and_tickers` expects (`pd.read_csv(... .csv)`), so no further changes needed there.
- If a file uses a different delimiter (e.g., fixed-width or something else), test with `pd.read_fwf(path_txt)` instead, or share a sample line for further tweaks.

### Step 3: Run and Verify
1. Ensure all `.txt` files are in place (e.g., `stooq/tickers/jp/tse etfs.txt`).
2. Rerun the notebook cells, starting from imports.
3. The loop should now process your local files without hitting the URL (no more `EmptyDataError`).
4. Check output: It will print the number of tickers per asset class (e.g., JP TSE ETFs might have ~100-200 entries).
5. After this, the price data processing (e.g., for files like `133a.jp.txt`) should proceed normally, as your example file is properly comma-separated and matches the expected format.
6. If errors occur during reading (e.g., mismatched delimiter), inspect the file and adjust `sep`.

### Additional Notes
- Price data files (like `133a.jp.txt`) are fine—they're comma-separated with a header, as shown, and the script handles them in `get_stooq_prices_and_tickers`.
- If Stooq changes formats, re-download symbols periodically.
- For large asset classes (e.g., US NYSE Stocks), the symbol list might be long—ensure your text editor handles it.
- If you encounter issues with a specific file, run a test in a separate cell: `df = pd.read_csv('stooq/tickers/jp/tse etfs.txt', sep='\s+', header=None); print(df.head())`. Adjust `sep` as needed.

In [20]:
metadata_dict = {
    ('jp', 'tse etfs'): 34,
    ('jp', 'tse stocks'): 32,
    ('us', 'nasdaq etfs'): 69,
    ('us', 'nasdaq stocks'): 27,
    ('us', 'nyse etfs'): 70,
    ('us', 'nyse stocks'): 28,
    ('us', 'nysemkt stocks'): 26
}

In [21]:
for (market, asset_class), code in metadata_dict.items():
    # Path to your manually downloaded .csv file (original remains unchanged)
    path_original = stooq_path / 'tickers' / market / f'{asset_class}.csv'
    
    # Read file line-by-line to handle variable spaces
    data = []
    with open(path_original, 'r') as file:
        for line in file:
            # Skip empty lines and header
            stripped_line = line.strip()
            if not stripped_line or stripped_line.startswith('<TICKER>'):
                continue
            # Split on the first sequence of spaces (or tab)
            parts = stripped_line.split(maxsplit=1)
            if len(parts) == 2:  # Ensure we have ticker and name
                ticker, name = parts
                data.append([ticker.strip('<>'), name.strip('<>')])
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['ticker', 'name'])
    
    # Clean up
    df = df.apply(lambda x: x.str.strip())
    df = df.drop_duplicates('ticker').dropna()
    
    print(market, asset_class, f'# tickers: {df.shape[0]:,.0f}')
    
    # Ensure directory exists
    path_original.parent.mkdir(parents=True, exist_ok=True)
    
    # Save processed version to a new file to avoid overwriting original
    path_processed = stooq_path / 'tickers' / market / f'{asset_class}_processed.csv'
    df.to_csv(path_processed, index=False)

jp tse etfs # tickers: 453
jp tse stocks # tickers: 3,953
us nasdaq etfs # tickers: 972
us nasdaq stocks # tickers: 4,318
us nyse etfs # tickers: 2,690
us nyse stocks # tickers: 3,558
us nysemkt stocks # tickers: 323


In [22]:
# for (market, asset_class), code in metadata_dict.items():
#     df = pd.read_csv(f'https://stooq.com/db/l/?g={code}', sep='        ').apply(lambda x: x.str.strip())
#     df.columns = ['ticker', 'name']
#     df = df.drop_duplicates('ticker').dropna()
#     print(market, asset_class, f'# tickers: {df.shape[0]:,.0f}')
#     path = stooq_path / 'tickers' / market
#     if not path.exists():
#         path.mkdir(parents=True)
#     df.to_csv(path / f'{asset_class}.csv', index=False)    

### Store price data in HDF5 format

To speed up loading, we store the price data in HDF format. The function `get_stooq_prices_and_symbols` loads data assuming the directory structure described above and takes the following arguments:
- frequency (see Stooq website for options as these may change; default is `daily`
- market (default: `us`), and 
- asset class (default: `nasdaq etfs`.

It removes files that do not have data or do not appear in the corresponding list of symbols.

In [23]:
def get_stooq_prices_and_tickers(frequency='daily',
                                 market='us',
                                 asset_class='nasdaq etfs'):
    prices = []
    
    # tickers = (pd.read_csv(stooq_path / 'tickers' / market / f'{asset_class}.csv'))
    tickers = (pd.read_csv(stooq_path / 'tickers' / market / f'{asset_class}_processed.csv'))

    if frequency in ['5 min', 'hourly']:
        parse_dates = [['date', 'time']]
        date_label = 'date_time'
    else:
        parse_dates = ['date']
        date_label = 'date'
    names = ['ticker', 'freq', 'date', 'time', 
             'open', 'high', 'low', 'close','volume', 'openint']
    
    usecols = ['ticker', 'open', 'high', 'low', 'close', 'volume'] + parse_dates
    path = stooq_path / 'data' / frequency / market / asset_class
    print(path.as_posix())
    files = path.glob('**/*.txt')
    for i, file in enumerate(files, 1):
        if i % 500 == 0:
            print(i)
        if file.stem not in set(tickers.ticker.str.lower()):
            print(file.stem, 'not available')
            file.unlink()
        else:
            try:
                df = (pd.read_csv(
                    file,
                    names=names,
                    usecols=usecols,
                    header=0,
                    parse_dates=parse_dates))
                prices.append(df)
            except pd.errors.EmptyDataError:
                print('\tdata missing', file.stem)
                file.unlink()

    prices = (pd.concat(prices, ignore_index=True)
              .rename(columns=str.lower)
              .set_index(['ticker', date_label])
              .apply(lambda x: pd.to_numeric(x, errors='coerce')))
    return prices, tickers

We'll be using US equities and ETFs in [Chapter 9](../09_time_series_models) and and Japanese equities in [Chapter 11](../11_decision_trees_random_forests). The following code collects the price data for the period 2000-2019 and stores it with the corresponding symbols in the global `assets.h5` store:

In [24]:
# load some Japanese and all US assets for 2000-2019
markets = {'jp': ['tse stocks'],
           'us': ['nasdaq etfs', 'nasdaq stocks', 'nyse etfs', 'nyse stocks', 'nysemkt stocks']
          }
frequency = 'daily'

idx = pd.IndexSlice
for market, asset_classes in markets.items():
    for asset_class in asset_classes:
        print(f'\n{asset_class}')
        prices, tickers = get_stooq_prices_and_tickers(frequency=frequency, 
                                                       market=market, 
                                                       asset_class=asset_class)
        
        prices = prices.sort_index().loc[idx[:, '2000': '2019'], :]
        names = prices.index.names
        prices = (prices
                  .reset_index()
                  .drop_duplicates()
                  .set_index(names)
                  .sort_index())
        
        print('\nNo. of observations per asset')
        print(prices.groupby('ticker').size().describe())
        key = f'stooq/{market}/{asset_class.replace(" ", "/")}/'
        
        print(prices.info(show_counts=True))
        
        prices.to_hdf(DATA_STORE, key + 'prices', format='t')
        
        print(tickers.info())
        tickers.to_hdf(DATA_STORE, key + 'tickers', format='t')


tse stocks
stooq/data/daily/jp/tse stocks
500
1000
1500
2000
2500
3000
3500

No. of observations per asset
count    3273.000000
mean     2827.595784
std      1174.797348
min         1.000000
25%      2179.000000
50%      3061.000000
75%      3621.000000
max      4905.000000
dtype: float64
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9254721 entries, ('1301.JP', Timestamp('2005-03-22 00:00:00')) to ('9997.JP', Timestamp('2019-12-30 00:00:00'))
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   open    9254721 non-null  float64
 1   high    9254721 non-null  float64
 2   low     9254721 non-null  float64
 3   close   9254721 non-null  float64
 4   volume  9254721 non-null  float64
dtypes: float64(5)
memory usage: 388.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3953 entries, 0 to 3952
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ticker  395